In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Introduction to `torch.compile`
===============================

**Author:** William Wen


`torch.compile` is the latest method to speed up your PyTorch code!
`torch.compile` makes PyTorch code run faster by JIT-compiling PyTorch
code into optimized kernels, all while requiring minimal code changes.

In this tutorial, we cover basic `torch.compile` usage, and demonstrate
the advantages of `torch.compile` over previous PyTorch compiler
solutions, such as
[TorchScript](https://pytorch.org/docs/stable/jit.html) and [FX
Tracing](https://pytorch.org/docs/stable/fx.html#torch.fx.symbolic_trace).

**Contents**

::: {.contents local=""}
:::

**Required pip Dependencies**

-   `torch >= 2.0`
-   `torchvision`
-   `numpy`
-   `scipy`
-   `tabulate`


NOTE: a modern NVIDIA GPU (H100, A100, or V100) is recommended for this
tutorial in order to reproduce the speedup numbers shown below and
documented elsewhere.


In [1]:
import torch
import warnings

gpu_ok = False
if torch.cuda.is_available():
    device_cap = torch.cuda.get_device_capability()
    if device_cap in ((7, 0), (8, 0), (9, 0)):
        gpu_ok = True

if not gpu_ok:
    warnings.warn(
        "GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower "
        "than expected."
    )

<ipython-input-1-dc0fbd3a2784>:11: UserWarning: GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
  warnings.warn(


Basic Usage
===========

`torch.compile` is included in the latest PyTorch. Running TorchInductor
on GPU requires Triton, which is included with the PyTorch 2.0 nightly
binary. If Triton is still missing, try installing `torchtriton` via pip
(`pip install torchtriton --extra-index-url "https://download.pytorch.org/whl/nightly/cu117"`
for CUDA 11.7).

Arbitrary Python functions can be optimized by passing the callable to
`torch.compile`. We can then call the returned optimized function in
place of the original function.


In [2]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
opt_foo1 = torch.compile(foo)
print(opt_foo1(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 1.1975e-01,  1.4721e+00, -8.2820e-01,  1.3972e+00,  1.3121e+00,
          1.2571e+00, -9.2285e-02,  5.8306e-01,  1.4178e+00,  1.3454e+00],
        [ 1.7640e+00,  1.2812e+00,  4.8281e-01,  1.8745e+00,  1.8813e+00,
          8.5921e-01, -1.2482e+00,  1.1343e+00,  1.0785e-01,  8.7893e-01],
        [ 3.4245e-01,  1.2222e+00,  3.9805e-01, -1.0765e+00,  1.9032e+00,
         -5.5640e-01,  3.7068e-01,  1.5084e+00,  2.7559e-01,  1.9343e+00],
        [ 1.5635e+00,  9.3303e-01,  1.8205e+00, -3.7439e-01,  1.2459e+00,
          1.6408e+00,  5.9428e-02,  1.6389e+00,  3.4642e-01,  1.5321e+00],
        [-5.9168e-01, -1.3909e+00,  1.7913e+00, -9.5276e-01,  1.6555e+00,
          1.5271e-01, -7.5891e-01, -1.2413e+00,  1.0304e+00, -2.1920e-01],
        [ 1.5160e-01, -3.1502e-01,  6.6586e-01,  6.0080e-01,  1.8747e+00,
          1.5920e-03,  1.2947e+00,  9.4726e-01,  5.5428e-01,  1.7116e+00],
        [ 4.5370e-01, -1.1541e-01, -1.5812e-01, -3.3844e-01,  2.5326e-01,
          7.8464e-01,  1.5855e+0

Alternatively, we can decorate the function.


In [3]:
t1 = torch.randn(10, 10)
t2 = torch.randn(10, 10)

@torch.compile
def opt_foo2(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
print(opt_foo2(t1, t2))

tensor([[ 1.4303e+00,  2.5748e-03,  5.5895e-01,  1.7998e-01,  5.7979e-01,
          1.4512e+00,  2.5365e-02,  1.8433e+00, -3.9247e-01,  7.8418e-01],
        [-2.4230e-01,  1.8632e+00,  8.2039e-02,  2.6518e-01,  7.4268e-01,
          2.7384e-01,  6.8096e-01,  1.1499e+00, -4.4376e-01, -1.5134e+00],
        [-6.0274e-02,  9.3553e-01,  8.2691e-02,  1.1409e+00,  1.6002e+00,
         -1.3179e+00,  1.4503e+00,  9.3492e-03,  1.2378e+00,  1.8979e+00],
        [ 5.9093e-01,  1.1635e+00, -1.4184e+00,  1.8358e+00,  6.5262e-01,
          1.5602e-02,  9.0455e-01,  1.5675e+00, -3.9138e-01,  1.9541e+00],
        [ 1.9425e+00,  7.9476e-01,  1.2377e+00,  1.5624e+00, -2.2739e-01,
         -4.4331e-01,  1.4920e+00,  1.0050e+00,  1.0931e+00,  1.9012e+00],
        [-1.7612e-01, -9.7802e-01,  4.5478e-02,  8.0646e-01,  1.0713e+00,
          1.0693e+00,  9.5822e-02,  1.0069e+00, -7.9539e-01,  1.2248e+00],
        [ 6.3823e-02,  1.2005e+00, -2.4176e-04, -3.1095e-01,  1.9462e+00,
          1.6128e-01,  1.1657e+0

We can also optimize `torch.nn.Module` instances.


In [4]:
t = torch.randn(10, 100)

class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))

mod = MyModule()
opt_mod = torch.compile(mod)
print(opt_mod(t))

tensor([[0.3258, 0.0000, 0.0000, 0.0198, 0.0194, 0.1515, 0.0000, 0.0000, 0.3577,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.4007, 0.0000, 0.4484, 0.0000, 0.0000,
         0.3945],
        [0.0607, 0.0000, 0.0000, 0.0000, 0.4184, 0.2607, 0.2385, 0.3233, 0.1778,
         0.0000],
        [0.8662, 0.2980, 0.0646, 0.3805, 0.0000, 0.6254, 0.3389, 0.0000, 0.0986,
         0.0000],
        [0.7208, 0.0000, 0.0000, 0.0000, 0.0737, 0.0000, 0.2437, 0.2410, 0.6259,
         0.0433],
        [0.1819, 0.3835, 0.3535, 0.0000, 0.0000, 0.8421, 0.1983, 0.0447, 0.2403,
         0.0000],
        [0.0000, 1.2544, 0.0000, 0.2715, 0.0000, 0.0000, 0.4720, 0.8252, 0.5106,
         0.0000],
        [0.7734, 0.9754, 0.0184, 0.0000, 0.0129, 0.5166, 0.6087, 0.4490, 0.1064,
         0.0187],
        [0.0000, 0.4849, 0.7646, 0.0551, 0.0000, 0.5123, 0.0000, 0.0000, 0.0000,
         1.0440],
        [0.3229, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4765,
         0.3014]], grad_fn=<

torch.compile and Nested Calls
==============================

Nested function calls within the decorated function will also be
compiled.


In [5]:
def nested_function(x):
    return torch.sin(x)

@torch.compile
def outer_function(x, y):
    a = nested_function(x)
    b = torch.cos(y)
    return a + b

print(outer_function(t1, t2))

tensor([[ 1.4303e+00,  2.5748e-03,  5.5895e-01,  1.7998e-01,  5.7979e-01,
          1.4512e+00,  2.5365e-02,  1.8433e+00, -3.9247e-01,  7.8418e-01],
        [-2.4230e-01,  1.8632e+00,  8.2039e-02,  2.6518e-01,  7.4268e-01,
          2.7384e-01,  6.8096e-01,  1.1499e+00, -4.4376e-01, -1.5134e+00],
        [-6.0274e-02,  9.3553e-01,  8.2691e-02,  1.1409e+00,  1.6002e+00,
         -1.3179e+00,  1.4503e+00,  9.3492e-03,  1.2378e+00,  1.8979e+00],
        [ 5.9093e-01,  1.1635e+00, -1.4184e+00,  1.8358e+00,  6.5262e-01,
          1.5602e-02,  9.0455e-01,  1.5675e+00, -3.9138e-01,  1.9541e+00],
        [ 1.9425e+00,  7.9476e-01,  1.2377e+00,  1.5624e+00, -2.2739e-01,
         -4.4331e-01,  1.4920e+00,  1.0050e+00,  1.0931e+00,  1.9012e+00],
        [-1.7612e-01, -9.7802e-01,  4.5478e-02,  8.0646e-01,  1.0713e+00,
          1.0693e+00,  9.5822e-02,  1.0069e+00, -7.9539e-01,  1.2248e+00],
        [ 6.3823e-02,  1.2005e+00, -2.4176e-04, -3.1095e-01,  1.9462e+00,
          1.6128e-01,  1.1657e+0

In the same fashion, when compiling a module all sub-modules and methods
within it, that are not in a skip list, are also compiled.


In [6]:
class OuterModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.inner_module = MyModule()
        self.outer_lin = torch.nn.Linear(10, 2)

    def forward(self, x):
        x = self.inner_module(x)
        return torch.nn.functional.relu(self.outer_lin(x))

outer_mod = OuterModule()
opt_outer_mod = torch.compile(outer_mod)
print(opt_outer_mod(t))

tensor([[0.2526, 0.1676],
        [0.3202, 0.0666],
        [0.0601, 0.1130],
        [0.4149, 0.0000],
        [0.2535, 0.0000],
        [0.1871, 0.0062],
        [0.2166, 0.0000],
        [0.1999, 0.1021],
        [0.1807, 0.0000],
        [0.3494, 0.0985]], grad_fn=<CompiledFunctionBackward>)


We can also disable some functions from being compiled by using
`torch.compiler.disable`. Suppose you want to disable the tracing on
just the `complex_function` function, but want to continue the tracing
back in `complex_conjugate`. In this case, you can use
`torch.compiler.disable(recursive=False)` option. Otherwise, the default
is `recursive=True`.


In [7]:
def complex_conjugate(z):
    return torch.conj(z)

@torch.compiler.disable(recursive=False)
def complex_function(real, imag):
    # Assuming this function cause problems in the compilation
    z = torch.complex(real, imag)
    return complex_conjugate(z)

def outer_function():
    real = torch.tensor([2, 3], dtype=torch.float32)
    imag = torch.tensor([4, 5], dtype=torch.float32)
    z = complex_function(real, imag)
    return torch.abs(z)

# Try to compile the outer_function
try:
    opt_outer_function = torch.compile(outer_function)
    print(opt_outer_function())
except Exception as e:
    print("Compilation of outer_function failed:", e)

/usr/local/lib/python3.11/dist-packages/torch/_inductor/lowering.py:1814: UserWarning: Torchinductor does not support code generation for complex operators. Performance may be worse than eager.
  warnings.warn(


tensor([4.4721, 5.8310])


Best Practices and Recommendations
==================================

Behavior of `torch.compile` with Nested Modules and Function Calls

When you use `torch.compile`, the compiler will try to recursively
compile every function call inside the target function or module inside
the target function or module that is not in a skip list (such as
built-ins, some functions in the torch.\* namespace).

**Best Practices:**

1\. **Top-Level Compilation:** One approach is to compile at the highest
level possible (i.e., when the top-level module is initialized/called)
and selectively disable compilation when encountering excessive graph
breaks or errors. If there are still many compile issues, compile
individual subcomponents instead.

2\. **Modular Testing:** Test individual functions and modules with
`torch.compile` before integrating them into larger models to isolate
potential issues.

3\. **Disable Compilation Selectively:** If certain functions or
sub-modules cannot be handled by [torch.compile]{.title-ref}, use the
[torch.compiler.disable]{.title-ref} context managers to recursively
exclude them from compilation.

4\. **Compile Leaf Functions First:** In complex models with multiple
nested functions and modules, start by compiling the leaf functions or
modules first. For more information see [TorchDynamo APIs for
fine-grained
tracing](https://pytorch.org/docs/stable/torch.compiler_fine_grain_apis.html).


Demonstrating Speedups
======================

Let\'s now demonstrate that using `torch.compile` can speed up real
models. We will compare standard eager mode and `torch.compile` by
evaluating and training a `torchvision` model on random data.

Before we start, we need to define some utility functions.


In [8]:
# Returns the result of running `fn()` and the time it took for `fn()` to run,
# in seconds. We use CUDA events and synchronization for the most accurate
# measurements.
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

# Generates random input and targets data for the model, where `b` is
# batch size.
def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )

N_ITERS = 10

from torchvision.models import densenet121
def init_model():
    return densenet121().to(torch.float32).cuda()

First, let\'s compare inference.

Note that in the call to `torch.compile`, we have the additional `mode`
argument, which we will discuss below.


In [9]:
model = init_model()

# Reset since we are using a different mode.
import torch._dynamo
torch._dynamo.reset()

model_opt = torch.compile(model, mode="reduce-overhead")

inp = generate_data(16)[0]
with torch.no_grad():
    print("eager:", timed(lambda: model(inp))[1])
    print("compile:", timed(lambda: model_opt(inp))[1])

eager: 0.9670557861328125


W0329 08:39:49.966000 284 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


compile: 163.899203125


Notice that `torch.compile` takes a lot longer to complete compared to
eager. This is because `torch.compile` compiles the model into optimized
kernels as it executes. In our example, the structure of the model
doesn\'t change, and so recompilation is not needed. So if we run our
optimized model several more times, we should see a significant
improvement compared to eager.


In [10]:
eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, eager_time = timed(lambda: model(inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, compile_time = timed(lambda: model_opt(inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

import numpy as np
eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 0.0334202880859375
eager eval time 1: 0.03018422317504883
eager eval time 2: 0.030175199508666994
eager eval time 3: 0.030146272659301757
eager eval time 4: 0.030174528121948242
eager eval time 5: 0.030146688461303712
eager eval time 6: 0.03016703987121582
eager eval time 7: 0.021141408920288086
eager eval time 8: 0.02022025680541992
eager eval time 9: 0.022903648376464844
~~~~~~~~~~
compile eval time 0: 0.8138728637695313
compile eval time 1: 0.020130207061767578
compile eval time 2: 0.02249932861328125
compile eval time 3: 0.021737472534179687
compile eval time 4: 0.02004035186767578
compile eval time 5: 0.019992576599121094
compile eval time 6: 0.020851167678833007
compile eval time 7: 0.019843936920166016
compile eval time 8: 0.019698944091796875
compile eval time 9: 0.019787776947021486
~~~~~~~~~~
(eval) eager median: 0.03015686416625977, compile median: 0.02008527946472168, speedup: 1.5014411036315471x
~~~~~~~~~~


And indeed, we can see that running our model with `torch.compile`
results in a significant speedup. Speedup mainly comes from reducing
Python overhead and GPU read/writes, and so the observed speedup may
vary on factors such as model architecture and batch size. For example,
if a model\'s architecture is simple and the amount of data is large,
then the bottleneck would be GPU compute and the observed speedup may be
less significant.

You may also see different speedup results depending on the chosen
`mode` argument. The `"reduce-overhead"` mode uses CUDA graphs to
further reduce the overhead of Python. For your own models, you may need
to experiment with different modes to maximize speedup. You can read
more about modes
[here](https://pytorch.org/get-started/pytorch-2.0/#user-experience).

You may might also notice that the second time we run our model with
`torch.compile` is significantly slower than the other runs, although it
is much faster than the first run. This is because the
`"reduce-overhead"` mode runs a few warm-up iterations for CUDA graphs.

For general PyTorch benchmarking, you can try using
`torch.utils.benchmark` instead of the `timed` function we defined
above. We wrote our own timing function in this tutorial to show
`torch.compile`\'s compilation latency.

Now, let\'s consider comparing training.


In [11]:
model = init_model()
opt = torch.optim.Adam(model.parameters())

def train(mod, data):
    opt.zero_grad(True)
    pred = mod(data[0])
    loss = torch.nn.CrossEntropyLoss()(pred, data[1])
    loss.backward()
    opt.step()

eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, eager_time = timed(lambda: train(model, inp))
    eager_times.append(eager_time)
    print(f"eager train time {i}: {eager_time}")
print("~" * 10)

model = init_model()
opt = torch.optim.Adam(model.parameters())
train_opt = torch.compile(train, mode="reduce-overhead")

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager train time 0: 0.6659010620117187
eager train time 1: 0.0919378890991211
eager train time 2: 0.08329830169677735
eager train time 3: 0.07142809295654297
eager train time 4: 0.0705227508544922
eager train time 5: 0.06850994873046876
eager train time 6: 0.07045471954345703
eager train time 7: 0.06908927917480469
eager train time 8: 0.06926710510253906
eager train time 9: 0.07071949005126953
~~~~~~~~~~


W0329 09:00:17.501000 284 torch/_logging/_internal.py:1089] [3/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored


compile train time 0: 356.26003125
compile train time 1: 9.414953125
compile train time 2: 0.08057942199707031
compile train time 3: 0.08069884490966797
compile train time 4: 0.06891385650634765
compile train time 5: 0.0493454704284668
compile train time 6: 0.05261932754516602
compile train time 7: 0.05103468704223633
compile train time 8: 0.050784351348876954
compile train time 9: 0.049178016662597655
~~~~~~~~~~
(train) eager median: 0.07062112045288085, compile median: 0.060766592025756835, speedup: 1.1621701678275298x
~~~~~~~~~~


Again, we can see that `torch.compile` takes longer in the first
iteration, as it must compile the model, but in subsequent iterations,
we see significant speedups compared to eager.

We remark that the speedup numbers presented in this tutorial are for
demonstration purposes only. Official speedup values can be seen at the
[TorchInductor performance
dashboard](https://hud.pytorch.org/benchmark/compilers).


Comparison to TorchScript and FX Tracing
========================================

We have seen that `torch.compile` can speed up PyTorch code. Why else
should we use `torch.compile` over existing PyTorch compiler solutions,
such as TorchScript or FX Tracing? Primarily, the advantage of
`torch.compile` lies in its ability to handle arbitrary Python code with
minimal changes to existing code.

One case that `torch.compile` can handle that other compiler solutions
struggle with is data-dependent control flow (the `if x.sum() < 0:` line
below).


In [12]:
def f1(x, y):
    if x.sum() < 0:
        return -y
    return y

# Test that `fn1` and `fn2` return the same result, given
# the same arguments `args`. Typically, `fn1` will be an eager function
# while `fn2` will be a compiled function (torch.compile, TorchScript, or FX graph).
def test_fns(fn1, fn2, args):
    out1 = fn1(*args)
    out2 = fn2(*args)
    return torch.allclose(out1, out2)

inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)

TorchScript tracing `f1` results in silently incorrect results, since
only the actual control flow path is traced.


In [13]:
traced_f1 = torch.jit.trace(f1, (inp1, inp2))
print("traced 1, 1:", test_fns(f1, traced_f1, (inp1, inp2)))
print("traced 1, 2:", test_fns(f1, traced_f1, (-inp1, inp2)))

traced 1, 1: True
traced 1, 2: False


<ipython-input-12-9380e8d561b2>:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:


FX tracing `f1` results in an error due to the presence of
data-dependent control flow.


In [14]:
import traceback as tb
try:
    torch.fx.symbolic_trace(f1)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "<ipython-input-14-bb92baf683e0>", line 3, in <cell line: 0>
    torch.fx.symbolic_trace(f1)
  File "/usr/local/lib/python3.11/dist-packages/torch/fx/_symbolic_trace.py", line 1307, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py", line 745, in _fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/fx/_symbolic_trace.py", line 843, in trace
    (self.create_arg(fn(*args)),),
                     ^^^^^^^^^
  File "<ipython-input-12-9380e8d561b2>", line 2, in f1
    if x.sum() < 0:
       ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/fx/proxy.py", line 549, in __bool__
    return self.tracer.to_bool(self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/fx/proxy.py", line 360, in to_boo

If we provide a value for `x` as we try to FX trace `f1`, then we run
into the same problem as TorchScript tracing, as the data-dependent
control flow is removed in the traced function.


In [15]:
fx_f1 = torch.fx.symbolic_trace(f1, concrete_args={"x": inp1})
print("fx 1, 1:", test_fns(f1, fx_f1, (inp1, inp2)))
print("fx 1, 2:", test_fns(f1, fx_f1, (-inp1, inp2)))

fx 1, 1: True
fx 1, 2: False


/usr/local/lib/python3.11/dist-packages/torch/fx/_symbolic_trace.py:906: UserWarning: Was not able to add assertion to guarantee correct input x to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  warnings.warn(


Now we can see that `torch.compile` correctly handles data-dependent
control flow.


In [16]:
# Reset since we are using a different mode.
torch._dynamo.reset()

compile_f1 = torch.compile(f1)
print("compile 1, 1:", test_fns(f1, compile_f1, (inp1, inp2)))
print("compile 1, 2:", test_fns(f1, compile_f1, (-inp1, inp2)))
print("~" * 10)

compile 1, 1: True
compile 1, 2: True
~~~~~~~~~~


TorchScript scripting can handle data-dependent control flow, but this
solution comes with its own set of problems. Namely, TorchScript
scripting can require major code changes and will raise errors when
unsupported Python is used.

In the example below, we forget TorchScript type annotations and we
receive a TorchScript error because the input type for argument `y`, an
`int`, does not match with the default argument type, `torch.Tensor`.


In [17]:
def f2(x, y):
    return x + y

inp1 = torch.randn(5, 5)
inp2 = 3

script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "<ipython-input-17-c25211324a06>", line 9, in <cell line: 0>
    script_f2(inp1, inp2)
RuntimeError: f2() Expected a value of type 'Tensor (inferred)' for argument 'y' but instead found type 'int'.
Inferred 'y' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 1
Value: 3
Declaration: f2(Tensor x, Tensor y) -> Tensor
Cast error details: Unable to cast 3 to Tensor


However, `torch.compile` is easily able to handle `f2`.


In [18]:
compile_f2 = torch.compile(f2)
print("compile 2:", test_fns(f2, compile_f2, (inp1, inp2)))
print("~" * 10)

compile 2: True
~~~~~~~~~~


Another case that `torch.compile` handles well compared to previous
compilers solutions is the usage of non-PyTorch functions.


In [19]:
import scipy
def f3(x):
    x = x * 2
    x = scipy.fft.dct(x.numpy())
    x = torch.from_numpy(x)
    x = x * 2
    return x

TorchScript tracing treats results from non-PyTorch function calls as
constants, and so our results can be silently wrong.


In [20]:
inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)
traced_f3 = torch.jit.trace(f3, (inp1,))
print("traced 3:", test_fns(f3, traced_f3, (inp2,)))

traced 3: False


<ipython-input-19-83cc6daff52d>:4: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = scipy.fft.dct(x.numpy())
<ipython-input-19-83cc6daff52d>:5: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)


TorchScript scripting and FX tracing disallow non-PyTorch function
calls.


In [21]:
try:
    torch.jit.script(f3)
except:
    tb.print_exc()

try:
    torch.fx.symbolic_trace(f3)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "<ipython-input-21-05ead7362108>", line 2, in <cell line: 0>
    torch.jit.script(f3)
  File "/usr/local/lib/python3.11/dist-packages/torch/jit/_script.py", line 1429, in script
    ret = _script_impl(
          ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/jit/_script.py", line 1205, in _script_impl
    fn = torch._C._jit_script_compile(
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_jit_internal.py", line 1239, in _try_get_dispatched_fn
    return boolean_dispatched.get(fn)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/weakref.py", line 452, in get
    return self.data.get(ref(key),default)
                         ^^^^^^^^
TypeError: cannot create weak reference to 'uarray._Function' object
Traceback (most recent call last):
  File "<ipython-input-21-05ead7362108>", line 7, in <cell line: 0>
    torch.fx.symbolic_trace(f3)
  File "/usr/local/lib/python3.

In comparison, `torch.compile` is easily able to handle the non-PyTorch
function call.


In [22]:
compile_f3 = torch.compile(f3)
print("compile 3:", test_fns(f3, compile_f3, (inp2,)))

compile 3: True


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/variables/functions.py:679: UserWarning: Graph break due to unsupported builtin scipy.fft._pocketfft.pypocketfft.PyCapsule.dct. This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind). If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround. If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use torch.compiler.allow_in_graph.
  torch._dynamo.utils.warn_once(msg)


TorchDynamo and FX Graphs
=========================

One important component of `torch.compile` is TorchDynamo. TorchDynamo
is responsible for JIT compiling arbitrary Python code into [FX
graphs](https://pytorch.org/docs/stable/fx.html#torch.fx.Graph), which
can then be further optimized. TorchDynamo extracts FX graphs by
analyzing Python bytecode during runtime and detecting calls to PyTorch
operations.

Normally, TorchInductor, another component of `torch.compile`, further
compiles the FX graphs into optimized kernels, but TorchDynamo allows
for different backends to be used. In order to inspect the FX graphs
that TorchDynamo outputs, let us create a custom backend that outputs
the FX graph and simply returns the graph\'s unoptimized forward method.


In [23]:
from typing import List
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

# Reset since we are using a different backend.
torch._dynamo.reset()

opt_model = torch.compile(init_model(), backend=custom_backend)
opt_model(generate_data(16)[0])

custom backend called with FX graph:
opcode         name                                                                                                         target                                                                                                       args                                                                                                                                                                                                                                                                                                                                                                                                                                                     kwargs
-------------  -----------------------------------------------------------------------------------------------------------  -----------------------------------------------------------------------------------------------------------  -------------------------------------------------

tensor([[ 0.3037, -0.0082,  0.0379,  ..., -0.0447,  0.0013,  0.0914],
        [ 0.2987, -0.1633,  0.0908,  ..., -0.0495,  0.0091,  0.0628],
        [ 0.1875, -0.0919,  0.0648,  ..., -0.1502, -0.1348,  0.0925],
        ...,
        [ 0.1598, -0.0562,  0.0716,  ..., -0.0834, -0.1840,  0.0873],
        [ 0.2035,  0.0720, -0.0003,  ..., -0.0438, -0.1182,  0.0596],
        [ 0.1365, -0.0288, -0.1056,  ..., -0.1428, -0.1586,  0.0363]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

Using our custom backend, we can now see how TorchDynamo is able to
handle data-dependent control flow. Consider the function below, where
the line `if b.sum() < 0` is the source of data-dependent control flow.


In [24]:
def bar(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

opt_bar = torch.compile(bar, backend=custom_backend)
inp1 = torch.randn(10)
inp2 = torch.randn(10)
opt_bar(inp1, inp2)
opt_bar(inp1, -inp2)

custom backend called with FX graph:
opcode         name    target                                                  args         kwargs
-------------  ------  ------------------------------------------------------  -----------  --------
placeholder    l_a_    L_a_                                                    ()           {}
placeholder    l_b_    L_b_                                                    ()           {}
call_function  abs_1   <built-in method abs of type object at 0x7cab4161ff00>  (l_a_,)      {}
call_function  add     <built-in function add>                                 (abs_1, 1)   {}
call_function  x       <built-in function truediv>                             (l_a_, add)  {}
call_method    sum_1   sum                                                     (l_b_,)      {}
call_function  lt      <built-in function lt>                                  (sum_1, 0)   {}
output         output  output                                                  ((x, lt),)   {}
cus

tensor([-8.8852e-01,  1.5986e-01,  2.7062e-01,  5.2301e-01, -2.0577e-02,
        -2.1541e-02,  1.4165e-01, -1.2209e-02, -3.1578e-04, -5.9661e-01])

The output reveals that TorchDynamo extracted 3 different FX graphs
corresponding the following code (order may differ from the output
above):

1.  `x = a / (torch.abs(a) + 1)`
2.  `b = b * -1; return x * b`
3.  `return x * b`

When TorchDynamo encounters unsupported Python features, such as
data-dependent control flow, it breaks the computation graph, lets the
default Python interpreter handle the unsupported code, then resumes
capturing the graph.

Let\'s investigate by example how TorchDynamo would step through `bar`.
If `b.sum() < 0`, then TorchDynamo would run graph 1, let Python
determine the result of the conditional, then run graph 2. On the other
hand, if `not b.sum() < 0`, then TorchDynamo would run graph 1, let
Python determine the result of the conditional, then run graph 3.

This highlights a major difference between TorchDynamo and previous
PyTorch compiler solutions. When encountering unsupported Python
features, previous solutions either raise an error or silently fail.
TorchDynamo, on the other hand, will break the computation graph.

We can see where TorchDynamo breaks the graph by using
`torch._dynamo.explain`:


In [25]:
# Reset since we are using a different backend.
torch._dynamo.reset()
explain_output = torch._dynamo.explain(bar)(torch.randn(10), torch.randn(10))
print(explain_output)

Graph Count: 2
Graph Break Count: 1
Op Count: 6
Break Reasons:
  Break Reason 1:
    Reason: generic_jump TensorVariable()
    User Stack:
      <FrameSummary file <ipython-input-24-973c2addf488>, line 3 in bar>
Ops per Graph:
  Ops 1:
    <built-in method abs of type object at 0x7cab4161ff00>
    <built-in function add>
    <built-in function truediv>
    <built-in function lt>
  Ops 2:
    <built-in function mul>
    <built-in function mul>
Out Guards:
  Guard 1:
    Name: ''
    Source: global
    Create Function: DEFAULT_DEVICE
    Guard Types: ['DEFAULT_DEVICE']
    Code List: ['utils_device.CURRENT_DEVICE == None']
    Object Weakref: None
    Guarded Class Weakref: None
  Guard 2:
    Name: ''
    Source: global
    Create Function: GRAD_MODE
    Guard Types: None
    Code List: None
    Object Weakref: None
    Guarded Class Weakref: None
  Guard 3:
    Name: "L['b'].sum"
    Source: local
    Create Function: HASATTR
    Guard Types: ['HASATTR']
    Code List: ["hasattr(L['b']

In order to maximize speedup, graph breaks should be limited. We can
force TorchDynamo to raise an error upon the first graph break
encountered by using `fullgraph=True`:


In [26]:
opt_bar = torch.compile(bar, fullgraph=True)
try:
    opt_bar(torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

Traceback (most recent call last):
  File "<ipython-input-26-20efd0e070e8>", line 3, in <cell line: 0>
    opt_bar(torch.randn(10), torch.randn(10))
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py", line 574, in _fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/convert_frame.py", line 1380, in __call__
    return self._torchdynamo_orig_callable(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/convert_frame.py", line 547, in __call__
    return _compile(
           ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/convert_frame.py", line 986, in _compile
    guarded_code = compile_inner(code, one_graph, hooks, transform)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/convert_frame.py", line 715, in compile_inner
    return

And below, we demonstrate that TorchDynamo does not break the graph on
the model we used above for demonstrating speedups.


In [27]:
opt_model = torch.compile(init_model(), fullgraph=True)
print(opt_model(generate_data(16)[0]))

tensor([[ 3.4662e-01,  2.4267e-01,  8.3899e-02,  ..., -3.3804e-01,
          4.1266e-01, -6.4666e-02],
        [ 5.7713e-01,  1.7173e-01, -1.0133e-02,  ..., -1.7636e-01,
          3.5752e-01, -4.3756e-04],
        [ 3.6297e-01,  1.9768e-01,  7.1413e-02,  ..., -3.5354e-01,
          4.3119e-01, -2.5554e-02],
        ...,
        [ 3.6005e-01,  2.3706e-01, -3.6880e-02,  ..., -4.7938e-01,
          4.2267e-01,  1.1918e-02],
        [ 3.5995e-01,  3.6805e-02, -2.0526e-02,  ..., -2.3683e-01,
          3.2674e-01, -5.3704e-02],
        [ 3.4839e-01,  1.7920e-01, -5.5693e-02,  ..., -2.6851e-01,
          2.5151e-01,  4.6939e-02]], device='cuda:0',
       grad_fn=<CompiledFunctionBackward>)


We can use `torch.export` (from PyTorch 2.1+) to extract a single,
exportable FX graph from the input PyTorch program. The exported graph
is intended to be run on different (i.e. Python-less) environments. One
important restriction is that the `torch.export` does not support graph
breaks. Please check [this
tutorial](https://pytorch.org/tutorials/intermediate/torch_export_tutorial.html)
for more details on `torch.export`.


Conclusion
==========

In this tutorial, we introduced `torch.compile` by covering basic usage,
demonstrating speedups over eager mode, comparing to previous PyTorch
compiler solutions, and briefly investigating TorchDynamo and its
interactions with FX graphs. We hope that you will give `torch.compile`
a try!
